# Texterfassung Steuerzentrale für Yijing

## 1. Imports und Konfiguration

In [ ]:
import os
import json
from pathlib import Path
from typing import List, Dict, Any

import google.generativeai as genai
import pandas as pd
from IPython.display import Markdown, display

### API-Konfiguration

In [ ]:
# Konfiguration des API-Schlüssels
api_key = os.environ.get("API_KEY")
if not api_key:
    raise ValueError("Die Umgebungsvariable 'API_KEY' ist nicht gesetzt.")
genai.configure(api_key=api_key)

### Modellwahl

In [ ]:
# Auswahl des Modells
model_speed = {
    'dumb': 'gemini-1.5-flash-8b',
    'fast': 'gemini-1.5-flash-latest',
    'clever': 'gemini-1.5-pro-latest',
    'experimental': 'gemini-exp-1121',
}
model_type = os.getenv("MODEL_TYPE", model_speed['clever'])
print("Verwendetes Modell:", model_type)

## 2. Datenimport

### Yijing Text einlesen

In [ ]:
yijing_txt_path = Path('yijing/resources/yijing.txt')

# Funktion zum Lesen des Yijing-Texts
def read_yijing_txt(path: Path = yijing_txt_path) -> str:
    with path.open('r', encoding='utf-8') as f:
        return f.read()

# Laden der Daten
yijing_txt = read_yijing_txt()
yijing_chapters = yijing_txt.split('\n')
print("Yijing-Text erfolgreich geladen.")
print("Anzahl der Kapitel:", len(yijing_chapters))

### Erste Einblicke in die Daten

In [ ]:
# Anzeige der ersten 500 Zeichen des Yijing-Texts
print(yijing_chapters[51])

## 3. Datenverarbeitung

### Strukturierter JSON-Text des ersten Kapitels

In [ ]:
yijing_processed = """
{
  "hexagram": {
    "name": "GUAI / DER DURCHBRUCH",
    "subtitle": "Die Entschlossenheit",
    "trigrams": {
      "above": {
        "name": "Dui",
        "attributes": "das Heitere, der See"
      },
      "below": {
        "name": "Kien",
        "attributes": "das Schöpferische, der Himmel"
      }
    },
    "meaning": {
      "description": "Das Zeichen bedeutet einerseits einen Durchbruch nach lange angesammelter Spannung, wie den Durchbruch eines geschwellten Flusses durch seine Dämme, wie einen Wolkenbruch. Auf menschliche Verhältnisse übertragen, ist es andererseits die Zeit, da allmählich die Gemeinen im Schwinden sind. Ihr Einfluß ist im Abnehmen, und durch eine entschlossene Aktion kommt eine Änderung der Verhältnisse zum Durchbruch.",
      "season": "dritter Monat (April-Mai)"
    }
  },
  "judgment": {
    "description": "Der Durchbruch. Entschlossen muß man am Hof des Königs die Sache bekanntmachen. Der Wahrheit gemäß muß sie verkündet werden. Gefahr! Man muß seine eigene Stadt benachrichtigen. Nicht fördernd ist es, zu den Waffen zu greifen. Fördernd ist es, etwas zu unternehmen.",
    "analysis": [
      "Leidenschaft und Vernunft können nicht zusammen bestehen, daher ist ein unbedingter Kampf notwendig.",
      "Entschlossenheit muß auf Stärke und Freundlichkeit beruhen.",
      "Kompromisse mit dem Schlechten sind nicht möglich.",
      "Der Kampf darf nicht direkt durch Gewalt geführt werden.",
      "Der Edle beginnt bei sich selbst, um das Böse zu entwaffnen."
    ]
  },
  "image": {
    "description": "Der See ist an den Himmel emporgestiegen: das Bild des Durchbruchs.",
    "lesson": "Der Edle spendet Reichtum nach unten hin und scheut es, bei seiner Tugend zu verweilen.",
    "warning": "Sammeln führt zu Zerstreuen; rechtzeitige Vorbereitung kann einem gewaltsamen Zusammenbruch vorbeugen."
  },
  "lines": [
    {
      "position": "Anfangs eine",
      "text": "Mächtig in den vorwärtsschreitenden Zehen. Geht man hin und ist der Sache nicht gewachsen, so macht man einen Fehler.",
      "interpretation": "Zu Beginn ist entschlossenes Voranschreiten schwierig. Blindes Draufgängertum führt zu unheilvollen Folgen."
    },
    {
      "position": "Neun auf zweitem Platz",
      "text": "Alarmruf. Abends und nachts Waffen. Fürchte nichts.",
      "interpretation": "Vorsicht und Wachsamkeit schützen vor Gefahren. Besonnenheit ist der rechte Weg zur Sicherheit."
    },
    {
      "position": "Neun auf drittem Platz",
      "text": "Mächtig in den Backenknochen zu sein bringt Unheil.",
      "interpretation": "Entschlossenheit ist notwendig, aber äußere Stärke zur falschen Zeit kann die Lage verschlimmern."
    },
    {
      "position": "Neun auf viertem Platz",
      "text": "An den Oberschenkeln ist keine Haut, und das Gehen fällt schwer.",
      "interpretation": "Eigensinn führt zu Konflikten. Würde man Ratschläge annehmen, könnte alles gutgehen."
    },
    {
      "position": "Neun auf fünftem Platz",
      "text": "Dem Unkraut gegenüber braucht es feste Entschlossenheit.",
      "interpretation": "Hindernisse müssen mit Entschlossenheit überwunden werden, ohne vom Weg abzukommen."
    },
    {
      "position": "Oben eine Sechs",
      "text": "Kein Ruf! Schließlich kommt Unheil.",
      "interpretation": "Nachlässigkeit beim Entfernen des Bösen führt zu erneutem Übel. Gründliche Arbeit ist notwendig."
    }
  ]
}
"""


In [ ]:
# Umwandlung des JSON-Textes in ein Python-Dictionary
yijing_data = json.loads(yijing_processed)
print("Erstes Kapitel erfolgreich verarbeitet.")

In [ ]:
#yijing_data

## 4. Funktionen zur Verarbeitung eines Hexagramms

In [ ]:
def process_hexagram(text: str, model: genai.GenerativeModel) -> Dict[str, Any]:
    """
    Verarbeitet einen Hexagramm-Text und gibt strukturierte Daten zurück.
    """
    prompt = HEXAGRAM_PROMPT + text
    result = model.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",
            response_schema=None  # Verwenden Sie None, wenn kein Schema benötigt wird
        ),
    )
    return result.candidates[0].content

## 5. Initialisierung des Modells

### Systemanweisung für das Modell

In [ ]:
HEXAGRAM_PROMPT = """
Du bist ein I Ging-Experte mit der Aufgabe, Hexagramm-Texte zu analysieren und in ein spezifisches JSON-Format zu überführen.

Wandle den Eingabetext in folgendes Format um:

{
  "hexagram": {
    "name": "Name des Hexagramms (z.B. 'GUAI / DER DURCHBRUCH')",
    "subtitle": "Untertitel oder Kernbedeutung", (z.B. 'Die Entschlossenheit')
    "trigrams": {
      "above": {
        "name": "Name des oberen Trigramms", (z.B. 'Dui')
        "attributes": "Eigenschaften des Trigramms" (z.B. 'das Heitere, der See')
      },
      "below": {
        "name": "Name des unteren Trigramms", (z.B. 'Kien')
        "attributes": "Eigenschaften des Trigramms" (z.B. 'das Schöpferische, der Himmel')
      }
    },
    "meaning": {
      "description": "Hauptbedeutung des Hexagramms", (z.B. 'Das Zeichen bedeutet...')
      "season": "Zugeordnete Jahreszeit" (z.B. 'dritter Monat (April-Mai)')
    }
  },
  "judgment": {
    "description": "Der Urteilstext", (z.B. 'Der Durchbruch...')
    "analysis": [
      "Liste von Analysepunkten zum Urteil" (z.B. 'Leidenschaft und Vernunft...')
    ]
  },
  "image": {
    "description": "Beschreibung des Bildes", (z.B. 'Der See ist an den Himmel emporgestiegen...')
    "lesson": "Lehre für den:die Edle:n", (z.B. 'Der:die Edle spendet Reichtum...')
    "warning": "Warnung oder zusätzliche Hinweise" # (z.B. 'Sammeln führt zu Zerstreuen...')
  },
  "lines": [
    {
      "position": "Position der Linie", (z.B. 'Anfangs eine Sechs')",
      "text": "Text der Linie", z.B. 'Mächtig in den vorwärtsschreitenden Zehen...'
      "interpretation": "Interpretation der Linie", z.B. 'Zu Beginn ist entschlossenes Voranschreiten...'
    }
  ]
}

Wichtige Hinweise:
1. Extrahiere alle relevanten Informationen aus dem Eingabetext
2. Behalte die originale Formulierung wo möglich bei
3. Stelle sicher, dass alle Pflichtfelder gefüllt sind
4. Das lines-Array muss genau 6 Einträge enthalten
5. Füge fehlende Informationen mit "Keine Information verfügbar" ein

Analysiere nun den folgenden Hexagramm-Text und gib ihn im spezifizierten JSON-Format zurück:
"""

In [ ]:
instruction = HEXAGRAM_PROMPT

In [ ]:
# Erstellen des Modells
model = genai.GenerativeModel(
    model_type,
    system_instruction=instruction
)

# Chat-Instanz starten
chat = model.start_chat()
print("Modell und Chat-Instanz erfolgreich initialisiert.")

## 6. Verarbeitung und Export

### Hexagramm verarbeiten und anzeigen

In [ ]:
def process_hexagram_text(hexagram_text: str) -> Dict[str, Any]:
    """
    Verarbeitet einen Hexagramm-Text und gibt strukturierte Daten zurück.

    Args:
        hexagram_text (str): Der Hexagramm-Text.

    Returns:
        dict: Strukturierte Daten des Hexagramms.
    """
    result = chat.send_message(hexagram_text)

    # Überprüfen, ob die Antwort Teile enthält
    if hasattr(result, 'parts') and len(result.parts) > 0:
        json_text = result.parts[0].text
        # Entfernen von Codeblöcken, falls vorhanden
        formatted_text = json_text.strip('```json\n').strip('```')
        
        try:
            # Laden des JSON in ein Python-Dictionary
            data = json.loads(formatted_text)
            display(Markdown("### Verarbeitetes Hexagramm"))
            display(json.dumps(data, indent=4, ensure_ascii=False))
            return data
        except json.JSONDecodeError:
            print("Fehler beim Parsen des JSON.")
    else:
        print("Keine Antwortteile gefunden.")
    
    return {}

In [ ]:
# Beispielhafte Nutzung mit dem ersten Kapitel
first_chapter_text = yijing_chapters[0] if 'yijing_chapters' in locals() else yijing_txt[:1000]
extracted_hexagram = process_hexagram_text(first_chapter_text)

### Speicherung der Daten als JSON

In [ ]:
def save_hexagram_json(hexagram_data: Dict[str, Any], output_dir: Path, filename: str) -> None:
    """
    Speichert die Hexagramm-Daten als JSON-Datei.

    Args:
        hexagram_data (dict): Strukturierte Hexagramm-Daten.
        output_dir (Path): Verzeichnis zum Speichern der JSON-Dateien.
        filename (str): Name der JSON-Datei.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = output_dir / filename
    with output_file.open('w', encoding='utf-8') as f:
        json.dump(hexagram_data, f, ensure_ascii=False, indent=4)
    print(f"Daten gespeichert unter {output_file}")

In [ ]:
# Speichern des extrahierten Hexagramms
if extracted_hexagram:
    save_hexagram_json(extracted_hexagram, Path('export/hexagram_json'), 'hexagram_01.json')

In [ ]:
# Erstellen des Modells
model = genai.GenerativeModel(
    model_type,
    system_instruction=instruction
)

# Chat-Instanz starten
chat = model.start_chat()
print("Modell und Chat-Instanz erfolgreich initialisiert.")

In [ ]:
i = 18

for chapter in [yijing_chapters[i-1]]:
    # Erstellen des Modells
    model = genai.GenerativeModel(
        model_type,
        system_instruction=instruction
    )

    # Chat-Instanz starten
    chat = model.start_chat()
    print("Modell und Chat-Instanz erfolgreich initialisiert.")

    print(f"Verarbeite Kapitel {i}...")
    extracted_hexagram = process_hexagram_text(chapter)
    print('Kapitel verarbeitet.')
    if extracted_hexagram:
        save_hexagram_json(extracted_hexagram, Path('export/hexagram_json'), f'hexagram_{i:02d}.json')
    i += 1

# XXXXXXXXXXX

In [ ]:
# load all json files in export/hexagram_json
# and store them in a list
hexagram_files = list(Path('export/hexagram_json').glob('*.json'))

hexagram_data = []

for file in hexagram_files:
    with file.open('r', encoding='utf-8') as f:
        data = json.load(f)
        hexagram_data.append(data)

hexagram_data


In [ ]:
df = pd.DataFrame(hexagram_data)
df.head()

In [ ]:
for col in df.columns:
    display(Markdown(f"### {col}"))
    display(df.loc[0, 'hexagram'])

In [ ]:
df['hexagram']

In [ ]:
# open import/hexagram_linkage.csv in pandas
linkage_df = (pd.read_csv('import/hexagram_linkage.csv', sep=';')
              .replace('ß', 'ss', regex=True)
              .replace('das durchbeissen', 'das durchbeißen', regex=True)
            )
linkage_df['Alter Name (Deutsch)'] = linkage_df['Alter Name (Deutsch)'].str.lower()
linkage_df

In [ ]:
# open import/hexagramme.csv in pandas
hexagramme_df = pd.read_csv('import/hexagramme.csv')
hexagramme_df

In [ ]:
df = pd.DataFrame(hexagram_data)

df_split = (df['hexagram'].apply(pd.Series)
            .replace('haftende', 'haftene', regex=True)
            .replace('das durchbeissen', 'das durchbeißen', regex=True)
            )

df_split['Alter Name (Deutsch)'] = df_split['name'].str.split(' / ').str[1].str.lower()

df_split = (df_split
            .replace('haftende', 'haftene', regex=True)
            .replace('das durchbeißen', 'das durchbeissen', regex=True)
            .replace('das haftene', 'das haftende', regex=True)
            .replace('die wiederkehrt', 'die wiederkehr', regex=True)
            )

df_merged = df_split.merge(linkage_df,
                           left_on='Alter Name (Deutsch)',
                           right_on='Alter Name (Deutsch)',
                           how='outer',
                           indicator=False)

df_merged = df_merged.merge(hexagramme_df, on='Nummer', how='outer', indicator=True)

#df_split = df_split['meaning'].apply(pd.Series)
#df_split = df_split['below'].apply(pd.Series)
#df_merged = df_merged[df_merged['_merge'] != 'both']

#df_merged = df_merged.iloc[:, :4].T

cols = [
    'Nummer',    
    'name',
    #'subtitle',
    #'trigrams',
    #'meaning',
    'Alter Name (Deutsch)',
    'Alter Code',
    'Neuer Code',
    'Vollständiger Name (Deutsch)',
    'Bezeichnung',
    #'Schriftzeichen',
    #'Pīnyīn',
    #'Line_Values',
    #'Title',
    'Hexagram',
    'Description',
    #'_merge'
    ]

#df_merged = df_merged.loc[:, cols].set_index('Nummer')
df_merged#.columns.to_list()
df_merged

In [ ]:
df = df_merged.copy()

for col in df.columns:
    print(f"Spalte: {col}")
    print("Inhalt:", df.loc[0, col])
    print()

In [ ]:
def export_df_to_json(df: pd.DataFrame, output_file: Optional[Path] = None) -> None:
    """
    Converts DataFrame of hexagram information to our standardized JSON schema format.
    
    Args:
        df: DataFrame containing hexagram information
        output_file: Optional path to save JSON file. If None, saves to default location.
    """
    if output_file is None:
        output_file = Path(__file__).parent / 'resources' / 'hexagrams.json'

    hexagrams = {}
    
    for _, row in df.iterrows():
        hexagram_number = int(row['Nummer'])
        
        # Process trigrams data
        trigrams = eval(row['trigrams']) if isinstance(row['trigrams'], str) else row['trigrams']
        
        # Process meaning data
        meaning_data = eval(row['meaning']) if isinstance(row['meaning'], str) else row['meaning']
        
        hexagram_data = {
            "number": hexagram_number,
            "names": {
                "chinese": row['Schriftzeichen'],
                "pinyin": row['Pīnyīn'],
                "german": row['Bezeichnung'],
                "english": row['Title'],
                "character": row['Schriftzeichen'],
                "old_german": row['Alter Name (Deutsch)'],
                "old_code": row['Alter Code'],
                "new_code": row['Neuer Code'],
                "full_german": row['Vollständiger Name (Deutsch)']
            },
            "trigrams": {
                "above": {
                    "name": trigrams['above']['name'],
                    "attributes": trigrams['above']['attributes']
                },
                "below": {
                    "name": trigrams['below']['name'],
                    "attributes": trigrams['below']['attributes']
                }
            },
            "technical": {
                "line_values": row['Line_Values'],
                "unicode_symbol": row['Hexagram']
            },
            "meaning": {
                "description": meaning_data['description'],
                "season": meaning_data.get('season'),
                "subtitle": row['subtitle']
            },
            "descriptions": {
                "main": row['Description'],
                "traditional": meaning_data['description']
            }
        }
        
        hexagrams[hexagram_number] = hexagram_data

    # Ensure the output directory exists
    output_file.parent.mkdir(parents=True, exist_ok=True)

    # Save to JSON file with proper formatting and encoding
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(hexagrams, f, ensure_ascii=False, indent=2)

    print(f"Successfully exported hexagram data to {output_file}")

In [ ]:
output_path = Path('export/hexagrams_test_01.json')

df = df_merged.copy()

export_df_to_json(df, output_path)

In [ ]:
# utils.py

import pandas as pd
from pathlib import Path
from typing import Dict, Optional
import json
from yijing.interpretations import HexagramInterpretation, YijingInterpretations

def load_hexagram_data() -> Dict[int, HexagramInterpretation]:
    """
    Loads and validates hexagram data from JSON storage.
    
    Returns:
        Dict[int, HexagramInterpretation]: Dictionary of validated hexagram interpretations
    
    Raises:
        FileNotFoundError: If hexagram data file is not found
        ValidationError: If data doesn't match expected schema
    """
    resources_dir = Path(__file__).parent / 'resources'
    hexagram_file = resources_dir / 'hexagrams.json'
    
    if not hexagram_file.exists():
        raise FileNotFoundError(f"Hexagram data file not found: {hexagram_file}")
        
    with open(hexagram_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    return {
        int(num): HexagramInterpretation(**hexagram_data)
        for num, hexagram_data in data.items()
    }

def export_df_to_json(df: pd.DataFrame, output_file: Optional[Path] = None) -> None:
    """
    Converts DataFrame of hexagram information to our JSON schema format.
    
    Args:
        df: DataFrame containing hexagram information
        output_file: Optional path to save JSON file. If None, saves to default location.
    """
    if output_file is None:
        output_file = Path(__file__).parent / 'resources' / 'hexagrams.json'
        
    # Transform DataFrame to our schema format
    hexagrams = {}
    for _, row in df.iterrows():
        hexagram_number = int(row['Nummer'])
        
        # Extract trigram information
        trigrams = eval(row['trigrams']) if isinstance(row['trigrams'], str) else row['trigrams']
        
        hexagrams[hexagram_number] = {
            "number": hexagram_number,
            "name_chinese": row['Schriftzeichen'],
            "name_pinyin": row['Pīnyīn'],
            "name_german": row['Vollständiger Name (Deutsch)'],
            "name_wilhelm": row['Alter Code'],
            "name_modern": row['Title'],
            "subtitle": row['subtitle'],
            "binary_sequence": row['Line_Values'],
            "unicode_symbol": row['Hexagram'],
            "upper_trigram": trigrams['above'],
            "lower_trigram": trigrams['below'] if 'below' in trigrams else None,
            "judgement": {
                "wilhelm_text": row['meaning']['description'] if isinstance(row['meaning'], dict) else ""
            }
        }
    
    # Save to JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(hexagrams, f, ensure_ascii=False, indent=2)

In [ ]:
df = df_merged.copy()  # Your existing DataFrame
export_df_to_json(df, 'export/hexagrams_test.json')

In [ ]:
# tools/convert_hexagram_data.py

import json
from pathlib import Path
from typing import Dict

def convert_hexagram_data(input_json: str, output_dir: Path) -> None:
    """
    Converts single JSON file to individual hexagram files.
    
    Args:
        input_json: Path to input JSON file
        output_dir: Directory for output files
    """
    # Create output directories
    hex_dir = output_dir / 'hexagrams'
    hex_dir.mkdir(parents=True, exist_ok=True)
    
    # Load input data
    with open(input_json) as f:
        data = json.load(f)
    
    # Process each hexagram
    binary_index = {}
    for num, hex_data in enumerate(data, start=1):
        # Add binary sequence if not present
        if 'binary_sequence' not in hex_data:
            hex_data['binary_sequence'] = format(num-1, '06b')
        
        # Update index
        binary_index[hex_data['binary_sequence']] = num
        
        # Save individual file
        output_file = hex_dir / f'{num:02d}.json'
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(hex_data, f, indent=2, ensure_ascii=False)
    
    # Save index
    with open(output_dir / 'index.json', 'w') as f:
        json.dump(binary_index, f, indent=2)


In [ ]:

if __name__ == '__main__':
    input_file = 'hexagrams.json'
    output_dir = Path('data')
    convert_hexagram_data(input_file, output_dir)

In [ ]:
df_split = df['hexagram'].apply(pd.Series)
    
i = 0

for index, row in df_split.iterrows():
    print(index, row['name'])

In [ ]:
# open import/hexagramme.csv in pandas
df_hexagramme = pd.read_csv('import/hexagramme.csv')
df_hexagramme.iloc[:, [0,1,-2]].to_markdown()

### Export der Linien in ein DataFrame

In [ ]:
def export_lines_to_dataframe(hexagram_data: Dict[str, Any]) -> pd.DataFrame:
    """
    Exportiert die Linien eines Hexagramms in ein pandas DataFrame.

    Args:
        hexagram_data (dict): Strukturierte Hexagramm-Daten.

    Returns:
        pd.DataFrame: DataFrame mit den Linieninformationen.
    """
    lines = hexagram_data.get('lines', [])
    lines_df = pd.DataFrame(lines)
    return lines_df

# Erstellen des DataFrames
lines_df = export_lines_to_dataframe(extracted_hexagram)
display(lines_df)

## 7. Zusätzliche Analysen (Optional)

### Beispiel: Analyse der Interpretationen

In [ ]:
if not lines_df.empty:
    from collections import Counter
    import re
    
    # Zusammenführen aller Interpretationen
    all_interpretations = ' '.join(lines_df['interpretation'].dropna().tolist())
    
    # Extrahieren von Wörtern
    words = re.findall(r'\w+', all_interpretations.lower())
    word_counts = Counter(words)
    
    # Anzeige der häufigsten Wörter
    common_words = word_counts.most_common(10)
    common_df = pd.DataFrame(common_words, columns=['Wort', 'Häufigkeit'])
    display(Markdown("### Häufigste Wörter in den Interpretationen"))
    display(common_df)

# Ende des Notebooks